- [previous file - EDA](2022-03-31_train-test_EDA.ipynb)
- [next tile - models with pre-tuned parameters and ensembles](2022-04-15_ensemble.ipynb)

## imports

In [4]:
from warnings import filterwarnings

import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
from sklearn.preprocessing import RobustScaler, LabelEncoder
from sklearn.ensemble import RandomForestRegressor, ExtraTreesRegressor, StackingRegressor
from catboost import CatBoostRegressor
from lightgbm import LGBMRegressor
import xgboost as xgb
from sklearn.linear_model import LinearRegression
from sklearn.neighbors import KNeighborsRegressor
import optuna
from sklearn.model_selection import train_test_split, cross_val_score
from sklearn.metrics import mean_absolute_percentage_error

from lib.model_related import *

sns.set()
filterwarnings("ignore")

## reading data

In [5]:
train_raw = pd.read_parquet("data/2022-04-08_train_pre-model.parquet")
test_raw = pd.read_parquet("data/2022-04-08_test_pre-model.parquet")

train_raw.shape, test_raw.shape

((115367, 30), (34686, 28))

In [6]:
def submit(hold_out: pd.DataFrame, model, name="submission"):
    preds = model.predict(hold_out)
    submission = pd.read_csv("data/sample_submission.csv")
    submission["price"] = preds
    submission.to_csv(f"{name}.csv", index=False)
    
    
def submit_log(hold_out: pd.DataFrame, model, name="submission"):
    preds = model.predict(hold_out)
    submission = pd.read_csv("data/sample_submission.csv")
    submission["price"] = np.exp(preds)
    submission.to_csv(f"{name}.csv", index=False)

## encoding

In [7]:
train_raw["train/test"] = "train"
test_raw["train/test"] = "test"

data = train_raw.append(test_raw)
data["ptc"].fillna("Оригинал", inplace=True)

data[data.select_dtypes("object").columns.tolist()] = data[
    data.select_dtypes("object").columns.tolist()
].astype(str)

for col in set(data.select_dtypes(exclude=("object")).columns) - {"price"}:
    data[col] = (
        RobustScaler().fit_transform(data[col].values.reshape(-1, 1)).reshape(-1, 1)
    )

for col in ["model_name"]:
    data[col] = LabelEncoder().fit_transform(data[col].astype("str"))

data = pd.get_dummies(
    data,
    columns=[
        "vehicle_transmission",
        "vendor",
        "brand",
        "fuel_type",
        "body_type",
        "color",
        "ptc",
        "drive",
        "wheel",
        "age_cat",
    ],
)

train = data.loc[data["train/test"] == "train"]

train_jane = train.loc[train["sample"] == "jane"]
train_sokolov = train.loc[train["sample"] == "sokolov"]
train_jane["price"] = train_jane["price"] * 0.86
train = train_jane.append(train_sokolov)

train.drop(columns=["sample", "description", "train/test"], inplace=True)
test = data.loc[data["train/test"] == "test"].drop(
    columns=["sample", "description", "train/test", "price"]
)

## modelling

In [8]:
X_train, X_valid, y_train, y_valid = train_test_split(train.drop(columns="price"), train["price"], random_state = 42, shuffle=True)
X_train.shape, y_train.shape, X_valid.shape, y_valid.shape

((86525, 112), (86525,), (28842, 112), (28842,))

In [11]:
lr = LinearRegression().fit(X_train, y_train)
knn = KNeighborsRegressor().fit(X_train, y_train)
lightgbm = LGBMRegressor(random_state=42, silent=True).fit(X_train, y_train)
catboost = CatBoostRegressor(random_state=42, silent=True).fit(X_train, y_train)
rf = RandomForestRegressor(random_state=42).fit(X_train, y_train)
rf_log = RandomForestRegressor(random_state=42).fit(X_train, np.log(y_train))
etr = ExtraTreesRegressor(random_state=42).fit(X_train, y_train)
etr_log = ExtraTreesRegressor(random_state=42).fit(X_train, np.log(y_train))

In [12]:
print("lr", mean_absolute_percentage_error(y_valid, lr.predict(X_valid)))
print("knn", mean_absolute_percentage_error(y_valid, knn.predict(X_valid)))
print("lightgbm", mean_absolute_percentage_error(y_valid, lightgbm.predict(X_valid)))
print("catboost", mean_absolute_percentage_error(y_valid, catboost.predict(X_valid)))
print("rf", mean_absolute_percentage_error(y_valid, rf.predict(X_valid)))
print("rf_log", mean_absolute_percentage_error(y_valid, np.exp(rf_log.predict(X_valid))))
print("etr", mean_absolute_percentage_error(y_valid, etr.predict(X_valid)))
print("etr_log", mean_absolute_percentage_error(y_valid, np.exp(etr.predict(X_valid))))

lr 0.7946185682561607
knn 0.16951330279094057
lightgbm 0.19102466397466208
catboost 0.1584574973807607
rf 0.13804863667174702


## dumb model submission

In [13]:
submit(test, lr, "lr")
submit(test, knn, "knn")
submit(test, lightgbm, "lightgbm")
submit(test, catboost, "catboost")
submit(test, rf, "rf")
submit(test, etr, "etr")
submit_log(test, rf_log, "rf_log")
submit_log(test, etr_log, "etr_log")

In [24]:
submit(test, etr, "etr")
submit_log(test, etr_log, "etr_log")